# This tutorial will show you how to find the suitable habitat range for Bristlecone pine using GeoPySpark in a GeoNotebook

### FYI: You are currently inside of a GeoNotebook.
GeoNotebook is an application that provides client/server enviroment with inteactive visualization and analysis capabilities using Jupyter notebook, GeoJS and other open source tools.
It was developed jointly developed by [Kitware](http://www.kitware.com/) and [NASA Ames](https://www.nasa.gov/centers/ames/home/index.html).

This tutorial will focus on GeoPySpark functionality, but you can find more resources and tutorials about GeoNotebooks [here](https://github.com/OpenGeoscience/geonotebook/tree/master/notebooks)


### Suitability analysis is a classic GIS case study that enables the combination of factors to return a desired result 
This tutorial sets the premise that you are interested in two factors for locating Bristlecone pines:
- Located between 3,000 and 4,000 meters
- Located on a south facing slope
     

In [ ]:
# GeoPySpark has lots of imports:
from pyspark import SparkContext
import geopyspark as gps

You will need to set up a spark context. To learn more about what that means take a look [here](https://spark.apache.org/docs/latest/programming-guide.html#initializing-spark)

In [ ]:
conf=gps.geopyspark_conf(appName="BristleConePine")
conf.set('spark.ui.enabled', True)
sc = SparkContext(conf = conf)

Retrieving an elevation .tif from the local file system:

In [ ]:
elev_rdd = gps.geotiff.get(
    layer_type='spatial', 
    uri='s3://geopyspark-demo/elevation/ca-elevation.tif')

In [ ]:
elev_rdd.count()

## Tile, reproject, pyramid:

In [ ]:
elev_tiled_rdd = elev_rdd.tile_to_layout(
    layout=gps.GlobalLayout(), 
    target_crs=3857)
elev_pyramided_rdd = elev_tiled_rdd.pyramid().cache()

Imports for creating a TMS server capable of serving layers with custom colormaps

In [ ]:
from geopyspark.geotrellis.color import ColorMap
from geopyspark.geotrellis.tms import TMSServer
from geonotebook.wrappers import TMSRasterData

In [ ]:
from geopyspark.geotrellis.color import get_colors_from_matplotlib
elev_histo        = elev_pyramided_rdd.get_histogram()
elev_colors       = get_colors_from_matplotlib('viridis', 100)
elev_color_map    = gps.ColorMap.from_histogram(elev_histo, elev_colors)

In [ ]:
elev_tms = gps.TMS.build(elev_pyramided_rdd, elev_color_map)

In [ ]:
M.set_center(-118, 38, 6)

In [ ]:
from geonotebook.wrappers import TMSRasterData
M.add_layer(TMSRasterData(elev_tms))

In [ ]:
M.remove_layer(M.layers[0])

Classify the elevation such that values of interest (between 3,000 and 4,000 meters) return a value of 1.

In [ ]:
# use: elev_reprojected_rdd
elev_reclass_pre = elev_tiled_rdd.reclassify({1000:2, 2000:2, 3000:2, 4000:1, 5000:2}, int)
elev_reclass_rdd = elev_reclass_pre.reclassify({1:1}, int)
elev_reclass_pyramid_rdd = elev_reclass_rdd.pyramid()

In [ ]:
elev_reclass_histo = elev_reclass_pyramid_rdd.get_histogram()

In [ ]:
#elev_reclass_color_map = ColorMap.from_histogram(sc, elev_reclass_histo, get_breaks(sc, 'Viridis', num_colors=100))
elev_reclass_color_map = gps.ColorMap.from_colors(
    breaks =[1], 
    color_list = [0xff000080])

In [ ]:
elev_reclass_tms = gps.TMS.build(elev_reclass_pyramid_rdd, elev_reclass_color_map)

In [ ]:
M.add_layer(TMSRasterData(elev_reclass_tms))

In [ ]:
M.remove_layer(M.layers[0])

Focal operation: aspect. To find south facing slopes

In [ ]:
from geopyspark.geotrellis.neighborhood import Square
from geopyspark.geotrellis.constants import Operation, Neighborhood

In [ ]:
elev_tiled_rdd.srdd.focal(
    Operation.ASPECT.value, 
    'square', 1.0, 0.0, 0.0).rdd().count()

In [ ]:
# square_neighborhood = Square(extent=1)
aspect_rdd = elev_tiled_rdd.focal(
    gps.Operation.SLOPE, 
    gps.Neighborhood.SQUARE, 1)

In [ ]:
aspect_pyramid_rdd       = aspect_rdd.pyramid()

In [ ]:
aspect_histo        = aspect_pyramid_rdd.get_histogram()
aspect_color_map    = gps.ColorMap.from_histogram(aspect_histo, get_colors_from_matplotlib('viridis', num_colors=256))
aspect_tms          = gps.TMS.build(aspect_pyramid_rdd, aspect_color_map)

In [ ]:
M.add_layer(TMSRasterData(aspect_tms))

In [ ]:
M.remove_layer(M.layers[0])

Reclassify values such that values between 120 and 240 degrees (south) have a value of 1

In [ ]:
aspect_reclass_pre  = aspect_rdd.reclassify({120:2, 240:1, 360: 2}, int)
aspect_reclass      = aspect_reclass_pre.reclassify({1:1}, int)

In [ ]:
aspect_reclass_pyramid_rdd       = aspect_reclass.pyramid()

In [ ]:
aspect_reclass_histo       = aspect_reclass_pyramid_rdd.get_histogram()
aspect_reclass_color_map   = gps.ColorMap.from_histogram(aspect_reclass_histo, get_colors_from_matplotlib('viridis', num_colors=256))
aspect_reclass_tms         = gps.TMS.build(aspect_reclass_pyramid_rdd, aspect_reclass_color_map)

In [ ]:
M.add_layer(TMSRasterData(aspect_reclass_tms))

In [ ]:
M.remove_layer(M.layers[0])

Now add the values togehter to find the suitable range:

In [ ]:
added = elev_reclass_pyramid_rdd + aspect_reclass_pyramid_rdd

In [ ]:
added_histo = added.get_histogram()
added_color_map = gps.ColorMap.from_histogram(added_histo, get_colors_from_matplotlib('viridis', num_colors=256))
added_tms = gps.TMS.build(added, added_color_map)

In [ ]:
M.add_layer(TMSRasterData(added_tms))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
v = elev_tiled_rdd.lookup(342,787)
plt.imshow(v[0]['data'][0])